# Práctica 4. Redes Convolucionales

In [13]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

from keras.datasets import mnist

num_classes = 10                # numeros [0-9]
input_shape = (28, 28, 1)       

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



AttributeError: module 'tensorflow.python.keras.utils' has no attribute 'to_categorical'

## Red Neuronal Convolucional 

In [9]:

# model = keras.Sequential(   
#     [
#         keras.Input(shape=input_shape),         
#         layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
#         layers.MaxPooling2D(pool_size=(2, 2)),
#         layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
#         layers.MaxPooling2D(pool_size=(2, 2)),
#         layers.Flatten(),
#         layers.Dropout(0.5),
#         layers.Dense(num_classes, activation="softmax"),
#     ]
# )

# otro constructor por capas (add/pop)
model = keras.Sequential(name="redensia")

#capas
model.add(keras.Input(shape=input_shape))                               # entrada -> dimensiones
model.add(layers.Conv2D(32, kernel_size=(3,3), activation="relu"))      # filtro 2D, normalmente se usan kernels 2x2/3x3, tamaño 32/64
model.add(layers.MaxPool2D(pool_size=(2,2)))                            # draw-sampling, agrupa para reducir dimensionalidad
model.add(layers.Conv2D(64, kernel_size=(3,3), activation="relu"))      # otro filtro
model.add(layers.MaxPool2D(pool_size=(2,2)))                            # ms agrupacion
model.add(layers.Flatten())                                             # reduce a un vector
model.add(layers.Dropout(0.5))                                          # porcentaje de neuronas que se apagan durante el aprendizaje, evita sobreaprendizaje
model.add(layers.Dense(num_classes, activation="softmax"))              # capa, en este caso es la salida tambien

model.summary()


Model: "redensia"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1600)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                160

In [10]:
batch_size = 128
epochs = 10

model.compile(
    loss="categorical_crossentropy",    # funcion de perdida
    optimizer="adam",                   # de optimizacion
    metrics=["accuracy"]                # metrica
)

model.fit(
    x_train,                        # conjunto de entrenamiento
    y_train,                        # de test
    batch_size=batch_size,          # tamaño lotes
    epochs=epochs,                  # epocas (repeticiones)
    validation_split=0.1            # controla el sobreaprendizaje
)

AttributeError: module 'tensorflow.python.distribute.input_lib' has no attribute 'DistributedDatasetInterface'

In [11]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])
print("Test error rate:", (1 - score[1])*100)

Test loss: 0.030098559334874153
Test accuracy: 0.989300012588501
Test error rate: 1.0699987411499023


## Ajuste de hiper-parámetros con Keras-Tuner

In [12]:
from tensorflow import keras
from tensorflow.python.keras import layers
import keras_tuner

# funcion que contruye una red convolucional
def call_existing_code(units, activation, dropout, lr):
    model = keras.Sequential()
    model.add(layers.Flatten())
    model.add(layers.Dense(units=units, activation=activation))
    if dropout:
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(10, activation="softmax"))
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

# contruye el modelo con varios parametros
def build_model(hp):
    units = hp.Int("units", min_value=32, max_value=512, step=32)
    activation = hp.Choice("activation", ["relu", "tanh"])
    dropout = hp.Boolean("dropout")
    lr = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    # call existing model-building code with the hyperparameter values.
    model = call_existing_code(
        units=units, activation=activation, dropout=dropout, lr=lr
    )
    return model


build_model(keras_tuner.HyperParameters())

In [2]:
(x, y), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x[:-10000]
x_val = x[-10000:]
y_train = y[:-10000]
y_val = y[-10000:]

x_train = np.expand_dims(x_train, -1).astype("float32") / 255.0
x_val = np.expand_dims(x_val, -1).astype("float32") / 255.0
x_test = np.expand_dims(x_test, -1).astype("float32") / 255.0

num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# usa las funciones antes definidas
# prueba en todas las configuraciones, como GridSearch
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=3,
    executions_per_trial=2,
    overwrite=True,
    directory="my_dir",
    project_name="helloworld",
)

tuner.search(x_train, y_train, epochs=2, validation_data=(x_val, y_val))

AttributeError: module 'tensorflow.python.keras' has no attribute 'datasets'

In [14]:
# Get the top 2 models.
models = tuner.get_best_models(num_models=2)
best_model = models[0]
# Build the model.
# Needed for `Sequential` without specified `input_shape`.
best_model.build(input_shape=(None, 28, 28))
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 module_wrapper (ModuleWrap  (None, 784)               0         
 per)                                                            
                                                                 
 module_wrapper_1 (ModuleWr  (None, 512)               401920    
 apper)                                                          
                                                                 
 module_wrapper_2 (ModuleWr  (None, 512)               0         
 apper)                                                          
                                                                 
 module_wrapper_3 (ModuleWr  (None, 10)                5130      
 apper)                                                          
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 

In [15]:
tuner.results_summary()

Results summary
Results in my_dir\helloworld
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
units: 512
activation: relu
dropout: True
lr: 0.0005653899495964376
Score: 0.972100019454956

Trial 1 summary
Hyperparameters:
units: 352
activation: tanh
dropout: True
lr: 0.0006361349306744992
Score: 0.9543000161647797

Trial 2 summary
Hyperparameters:
units: 160
activation: tanh
dropout: True
lr: 0.00018840040300474622
Score: 0.9294500052928925


### Re-entranando el modelo

In [16]:
# Get the top 2 hyperparameters.
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
model = build_model(best_hps[0])
# Fit with the entire dataset.
x_all = np.concatenate((x_train, x_val))
y_all = np.concatenate((y_train, y_val))
model.fit(x=x_all, y=y_all, epochs=1)

score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])
print("Test error rate:", (1 - score[1])*100)



1875/1875 [==============================] - 6s 3ms/step - loss: 0.2578 - accuracy: 0.9268
Test loss: 0.12047871947288513
Test accuracy: 0.9634000062942505
Test error rate: 3.659999370574951


In [17]:
tuner.oracle.trials['0'].metrics.metrics['val_loss'].get_statistics()


{'min': 0.0963003896176815,
 'max': 0.0963003896176815,
 'mean': 0.0963003896176815,
 'median': 0.0963003896176815,
 'var': 0.0,
 'std': 0.0}

## Ajuste de parámetros con apoyo de Scikit-Learn

In [1]:
import keras
from tensorflow.python.keras import layers
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV, RepeatedKFold
import numpy as np

input_shape = (28, 28, 1)
num_classes = 10

(x, y), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x[:-10000]
x_val = x[-10000:]
y_train = y[:-10000]
y_val = y[-10000:]

x_train = np.expand_dims(x_train, -1).astype("float32") / 255.0
x_val = np.expand_dims(x_val, -1).astype("float32") / 255.0
x_test = np.expand_dims(x_test, -1).astype("float32") / 255.0

num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



 
# Function to create model, required for KerasClassifier
def create_model(kernel_size=(2,2), pool_size=(2,2), rate=0.5):
    # create model
    model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=kernel_size, activation="relu"),
        layers.MaxPooling2D(pool_size=pool_size),
        layers.Conv2D(64, kernel_size=kernel_size, activation="relu"),
        layers.MaxPooling2D(pool_size=pool_size),
        layers.Flatten(),
        layers.Dropout(rate=rate),
        layers.Dense(num_classes, activation="softmax"),
    ])
    # Compile model
    
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model
 
# fix random seed for reproducibility
seed = 123
np.random.seed(seed)

# create model
model = KerasClassifier(model=create_model, verbose=0)
print(model.get_params().keys())

kernel_sizes = [(2,2), (3,3)]
pool_sizes = [(2,2)]
rates = [0.5, 0.75]

epochs = [50]
batches = [128]

param_grid = dict(model__kernel_size=kernel_sizes,  ##model__JUAN por parametros raros nose
                  epochs=epochs, 
                  batch_size=batches, 
                  model__pool_size=pool_sizes,
                  model__rate=rates)

cv = RepeatedKFold(n_splits=3, n_repeats=2, random_state=123)

search = GridSearchCV(estimator=model, 
                      param_grid=param_grid, 
                      scoring="accuracy", 
                      cv=cv, 
                      n_jobs=10)

grid_result = search.fit(X=x_train, y=y_train)

# ahora llevar a pandas y funciones estadisiticas de practica 3
# esto tarda muncho por las epocas y por que no hay early stopping

# jogar ---
# kernel de 3x3 dpm
# lotes 128, 256, 512
# learning rate 0.001

# del predict --> hacer un script quitando todo lo que no sea un numero: CONSEGUIR SUPERNUMERO


dict_keys(['model', 'build_fn', 'warm_start', 'random_state', 'optimizer', 'loss', 'metrics', 'batch_size', 'validation_batch_size', 'verbose', 'callbacks', 'validation_split', 'shuffle', 'run_eagerly', 'epochs', 'class_weight'])


KeyboardInterrupt: 

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
y_train